In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/Users/onivron/Downloads/weatherAUS.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
 data.isna().sum()

In [ ]:
data[data['Cloud9am'].isna()].head

In [ ]:
for i in range(data.shape[1]):
    print('Column: ' + data.columns[i]  + ' Unique values: ' + str(pd.unique(data.iloc[: , i])) )

In [ ]:
data.shape

In [ ]:
# percentage of missing values
data.isna().sum() / data.shape[0]

In [ ]:
pd.unique(data['Location'])

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data = data.drop('RISK_MM', axis=1)

In [ ]:
data.columns

In [ ]:
import numpy as np 

In [ ]:
## Assume cloud = nan means 0
data[data['Cloud9am'] == np.nan ] = 0 

In [ ]:
data[data['Cloud3pm'] == np.nan ] = 0 

In [ ]:
# lets remove all columns with greater than 35% missing values 

In [ ]:
list(data.columns)

In [ ]:
df = data

In [ ]:
for column in list(data.columns):
    if (df[df[column].isna()].shape[0] / df.shape[0]) > 0.35 : 
        print ('+1')
        print(column)
        df = df.drop(column, axis =1)

In [ ]:
# RiSK MM would give hinsight so let's remove it
df = df.drop('RISK_MM', axis =1)

In [ ]:
df.head()

In [ ]:
 df.isna().sum()

In [ ]:
# to fill Rain Today let's find out the mode for that month of this column

In [ ]:
# lets add a month column

In [ ]:
(df.Date[1]).split('-')[1]

In [ ]:
df['Date'].apply(lambda x: x.split('-'))

In [ ]:
df['Month'] = df['Date'].apply(lambda x: x.split('-')[1])

In [ ]:
df['Year'] = df['Date'].apply(lambda x: x.split('-')[0])

In [ ]:
df['Day'] = df['Date'].apply(lambda x: x.split('-')[2])

In [ ]:
df.Year.unique()

In [ ]:
df[['Year','Month','RainTomorrow']]

In [ ]:
df[['Year','Month','RainTomorrow']].groupby(['Year','Month']).mode()

In [ ]:
import scipy.stats

In [ ]:
df.columns

In [ ]:
df[['Location','Year','Month','RainToday']]

In [ ]:
rain_today_mode = df[['Location','Year','Month','RainToday']][~df['RainToday'].isna()].groupby(['Location', 'Year','Month']).agg(lambda x: scipy.stats.mode(x)[0])['RainToday'].reset_index()

In [ ]:
df[df['RainToday'].isna()]['RainToday']

In [ ]:
df2 = pd.merge(df, rain_today_mode, on=['Location','Year','Month'])

In [ ]:
df2

In [ ]:
df2[df2['RainToday_x'].isna()]['RainToday_y']

In [ ]:
df2.loc[df2['RainToday_x'].isna(), 'RainToday_x'] = df2[df2['RainToday_x'].isna()]['RainToday_y']

In [ ]:
df2['RainToday_x'].unique()

In [ ]:
df3 = df2.drop('RainToday_y', axis=1)

In [ ]:
# lets check if rainToday is always missing together with Rainfall ? 

In [ ]:
df.loc[df['RainToday'].isna(), 'Rainfall'].shape

In [ ]:
# Yes all are missing together! \
#which means whenever we took the mode to be No we should insert 0 otherwise we will take the monthly average? 

In [ ]:
rainfall_mean = df[['Location','Year','Month','Rainfall']][~df['Rainfall'].isna()].groupby(['Location', 'Year','Month']).mean()['Rainfall'].reset_index()

In [ ]:
rainfall_mean

In [ ]:
 df4 = pd.merge(df3, rainfall_mean, on=['Location','Year','Month'])

In [ ]:
type(df4['RainToday_x'][1])

In [ ]:
df4

In [ ]:
df4.columns

In [ ]:
a = df4[['Rainfall_x', 'Rainfall_y', 'RainToday_x']].apply(lambda x: x['Rainfall_y'] if x['RainToday_x'] == 'Yes' else 0 , axis =1)

In [ ]:
# assign 0 to rainfall whenever Rain today = No else assign mean

In [ ]:
df4['Rainfall_x'] = df4[['Rainfall_x', 'Rainfall_y', 'RainToday_x']].apply(lambda x: x['Rainfall_y'] if x['RainToday_x'] == 'Yes' else 0 , axis =1)

In [ ]:
# drop rainfall y

In [ ]:
df5 = df4.drop('Rainfall_y', axis = 1)

In [ ]:
# let's pick the mode also for the wind direction per location 

In [ ]:
wind_dir_9 = df[['Location','Year','Month','WindDir9am']][~df['WindDir9am'].isna()].groupby(['Location', 'Year','Month']).agg(lambda x: scipy.stats.mode(x)[0])['WindDir9am'].reset_index()

In [ ]:
wind_dir_9

In [ ]:
wind_dir_9.WindDir9am.unique()

In [ ]:
wind_dir_3pm = df[['Location','Year','Month','WindDir3pm']][~df['WindDir3pm'].isna()].groupby(['Location', 'Year','Month']).agg(lambda x: scipy.stats.mode(x)[0])['WindDir3pm'].reset_index()

In [ ]:
 df6 = pd.merge(df5, wind_dir_9, on=['Location','Year','Month'])

In [ ]:
df6

In [ ]:
df6.loc[df6['WindDir9am_x'].isna(), 'WindDir9am_x'] 

In [ ]:
df6.loc[df6['WindDir9am_x'].isna(), 'WindDir9am_x'] = df6[df6['WindDir9am_x'].isna()]['WindDir9am_y']

In [ ]:
 df7 = pd.merge(df6, wind_dir_3pm, on=['Location','Year','Month'])

In [ ]:
df7.loc[df7['WindDir3pm_x'].isna(), 'WindDir3pm_x'] = df7[df7['WindDir3pm_x'].isna()]['WindDir3pm_y']

In [ ]:
# let's check there are no more winDir nan

In [ ]:
list(df7.columns[(df7.isna().sum() > 0)])

In [ ]:
df7[df7.isna().sum() > 0]

In [ ]:
# let's drop the y columnms of WindDir

In [ ]:
df8 = df7.drop(['WindDir3pm_y', 'WindDir9am_y'], axis=1)

In [ ]:
df8['WindGustDir'].unique()

In [ ]:
WindGustDir = df[['Location','Year','Month','WindGustDir']][~df['WindGustDir'].isna()].groupby(['Location', 'Year','Month']).agg(lambda x: scipy.stats.mode(x)[0])['WindGustDir'].reset_index()

In [ ]:
 df9 = pd.merge(df8, WindGustDir, on=['Location','Year','Month'])

In [ ]:
df9.loc[df9['WindGustDir_x'].isna(), 'WindGustDir_x'] = df9[df9['WindGustDir_x'].isna()]['WindGustDir_y']

In [ ]:
df9

In [ ]:
# lets look at the distribution of columns with missing values per location per month

In [ ]:
dist_outlook = df9.groupby(['Location', 'Month', 'Day']).mean().reset_index()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(1, 13):
    if i < 10:
        month = '0' + str(i)
    else: 
        month = str(i)
    plt.hist(dist_outlook.loc[(dist_outlook['Location']=='Adelaide') & (dist_outlook['Month'] == month) , 'MinTemp'])
    plt.show()

In [ ]:
import seaborn as sns
corr_plots = sns.PairGrid(df9)
corr_plots.map(plt.scatter);

In [ ]:
# let's choose the median for all other values to avoid surprises from outliers

In [ ]:
for column in list(df9.columns[(df9.isna().sum() > 0)]):
    median = df[['Location','Year','Month',column]][~df[column].isna()].groupby(['Location', 'Year','Month']).median().reset_index()
    df9 = pd.merge(df9, median, on=['Location','Year','Month'])
    df9.loc[df9[column + '_x'].isna(), column + '_x'] = df9[df9[column + '_x'].isna()][column + '_y']
    
    

In [ ]:
# Do we still have missing values? 

In [ ]:
sum(df9.isna().sum())

In [ ]:
# Yayy No missing values

In [ ]:
# let's remove any column ending with _y 

In [ ]:
print (~df9.columns.str.endswith('_y'))

In [ ]:
df9 = df9.loc[:,~df9.columns.str.endswith('_y')]

In [ ]:
df9.columns

In [ ]:
# Next step would be to get rid of Date column since we have the year, Month, Day columns (we need to turn them to numeric values - they don't need to be categorical under a tree model)
# and then turn all categorical to dummies

In [ ]:
pd.get_dummies(df8[['Location', 'WindDir9am_x']])

In [ ]:
dummies =  pd.get_dummies(df9[['Location', 'WindGustDir_x', 'WindDir9am_x', 'WindDir3pm_x', 'RainToday_x']], ['Location', 'WindGustDir_x', 'WindDir9am_x', 'WindDir3pm_x', 'RainToday_x'])

In [ ]:
df9 = df9.drop(['Location', 'WindGustDir_x', 'WindDir9am_x', 'WindDir3pm_x', 'RainToday_x'], axis =1)

In [ ]:
df9 = pd.concat([df9, dummies], axis=1)

In [ ]:
df9['Year'] = df9['Year'].astype(int)

In [ ]:
df9 = df9.drop('Date', axis =1)

In [ ]:
index_split = df9['Year'] > 2014

In [ ]:
# let's split the data according to year

In [ ]:
index_split

In [ ]:
# let's turn the RainTomorrow column to numeric 0/1

In [ ]:
df9['RainTomorrow'] = df9['RainTomorrow'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
tr_x, tr_y, te_x, te_y = df9.loc[index_split, df9.columns != 'RainTomorrow'], df9.loc[index_split, 'RainTomorrow'], df9.loc[~index_split, df9.columns != 'RainTomorrow'], df9.loc[~index_split,'RainTomorrow'] 

In [ ]:
from sklearn import tree, ensemble

In [ ]:
clf = tree.DecisionTreeClassifier()

In [ ]:
clf = clf.fit(tr_x, tr_y)

In [ ]:
y_hat = clf.predict(te_x)

In [ ]:
np.sum(y_hat)/ len(y_hat)

In [ ]:
# let's calculate the misclassification error 

In [ ]:
np.where((y_hat == te_y) == False)[0].shape[0]/ te_y.shape[0]

In [ ]:
clf = ensemble.RandomForestClassifier()

In [ ]:
clf = clf.fit(tr_x, tr_y)

In [ ]:
y_hat = clf.predict(te_x)

In [ ]:
np.where((y_hat == te_y) == False)[0].shape[0]/ te_y.shape[0]